# Launch jobs
Code to create a batch script for launching jobs on cori \
Sep 1, 2020

In [14]:
import os
import glob
import time
import subprocess as sp
import numpy as np

In [15]:
curr_dir=os.getcwd()
print(curr_dir)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_data


In [16]:
img_size=128
code_dirs={'128':'/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/1_train/main_code/',
         '512':'/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/1_train/512_sqr/'}

results_loc='/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/{0}square/'.format(img_size)

In [17]:
# ls /global/cscratch1/sd/vpa/proj/cosmogan/results_dir/512square/20200901_200005_bsize50_30eph/chkpt/trainer0/sgd.shared.training.epoch.30.step.13500/model0/

In [18]:
# glob.glob(results_loc+'*')

In [19]:
### Default dictionary
dict_pars={'epochs':5, 'seed':282, 'nodes':1, 'procs':8,'job_name':'gen_img',
#            'spec_loss':'',
           'spec_loss':'--spec_loss','mcr':'',
           'lambda_spec':0.5,'learn_rate':0.002,
          'time':'00:15:00', 'batchsize':1000, 'step_interval':1}

fldr,epoch,step='20201202_200004_bsize256_scale0.5',16,14635
data_dir='/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/{3}square/{0}/chkpt/trainer0/sgd.shared.training.epoch.{1}.step.{2}/model0/'.format(fldr,epoch,step,img_size)

dict_pars['data_dir']=data_dir
dict_pars['code_dir']=code_dirs[str(img_size)]
print(dict_pars)


{'epochs': 5, 'seed': 282, 'nodes': 1, 'procs': 8, 'job_name': 'gen_img', 'spec_loss': '--spec_loss', 'mcr': '', 'lambda_spec': 0.5, 'learn_rate': 0.002, 'time': '00:15:00', 'batchsize': 1000, 'step_interval': 1, 'data_dir': '/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/20201202_200004_bsize256_scale0.5/chkpt/trainer0/sgd.shared.training.epoch.10.step.9475/model0/', 'code_dir': '/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/1_train/main_code/'}


In [20]:
assert os.path.isdir(dict_pars['data_dir'])

In [21]:
bash_strg='''#!/bin/bash
#################
#SBATCH --nodes=1
#SBATCH --qos=regular
#SBATCH --output=slurm-%x-%j.out
#SBATCH --constraint=gpu
#SBATCH --account=m3363
#SBATCH --ntasks-per-node=8
#SBATCH --cpus-per-task=8
#SBATCH --gpus-per-task=1
#SBATCH --time={time}
#SBATCH --job-name={job_name}

echo "--start date" `date` `date +%s`
echo '--hostname ' $HOSTNAME

### Initial setup
module purge
module load cgpu
module load modules/3.2.11.4 gcc/8.3.0 cuda/10.2.89 mvapich2/2.3.2 cmake/3.14.4 python3/3.7-anaconda-2019.10

module use /global/cfs/cdirs/m3363/lbann/tom_lbann_install/etc/modulefiles
module load lbann

export MKL_THREADING_LAYER=GNU
#export SPACK_ROOT=/global/cfs/cdirs/m3363/lbann/spack.git
#export SPACK_ROOT=/global/cfs/cdirs/m3363/lbann/tom_spack
#$SPACK_ROOT/share/spack/setup-env.sh

### Brian's installation
export SPACK_ROOT=/global/cfs/cdirs/m3363/lbann/spack.git; . $SPACK_ROOT/share/spack/setup-env.sh
spack env activate -p lbann-dev-skylake_avx512

module use /global/cfs/cdirs/m3363/lbann/lbann.git/build/gnu.Release.cgpu.nersc.gov/install/etc/modulefiles/                                                                                                       
module load lbann

export MV2_ENABLE_AFFINITY=0
export MV2_USE_CUDA=1
export IBV_FORK_SAFE=1

### Run the main code
code_dir={code_dir}
data_dir={data_dir}                                                 

python $code_dir/test_exagan.py --seed {seed} -b {batchsize} -stp {step_interval} {mcr} {spec_loss} --lambda_spec {lambda_spec} -lr {learn_rate} --nodes {nodes} --procs {procs} --suffix {job_name} --epochs {epochs} -dr $data_dir 
echo "--end date" `date` `date +%s`
'''.format(**dict_pars)

In [22]:
# bash_strg

In [23]:
### Write bash file
staging_loc='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_data/'
fname='batch_gen_images.sh'
filename=staging_loc+fname
print(filename)
with open (filename,'w') as f:
    f.write(bash_strg)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_data/batch_gen_images.sh


In [24]:
%%bash -s "$filename" ## Use python variable in bash
cat $1
chmod +x $1

#!/bin/bash
#################
#SBATCH --nodes=1
#SBATCH --qos=regular
#SBATCH --output=slurm-%x-%j.out
#SBATCH --constraint=gpu
#SBATCH --account=m3363
#SBATCH --ntasks-per-node=8
#SBATCH --cpus-per-task=8
#SBATCH --gpus-per-task=1
#SBATCH --time=00:15:00
#SBATCH --job-name=gen_img

echo "--start date" `date` `date +%s`
echo '--hostname ' $HOSTNAME

### Initial setup
module purge
module load cgpu
module load modules/3.2.11.4 gcc/8.3.0 cuda/10.2.89 mvapich2/2.3.2 cmake/3.14.4 python3/3.7-anaconda-2019.10

module use /global/cfs/cdirs/m3363/lbann/tom_lbann_install/etc/modulefiles
module load lbann

export MKL_THREADING_LAYER=GNU
#export SPACK_ROOT=/global/cfs/cdirs/m3363/lbann/spack.git
#export SPACK_ROOT=/global/cfs/cdirs/m3363/lbann/tom_spack
#$SPACK_ROOT/share/spack/setup-env.sh

### Brian's installation
export SPACK_ROOT=/global/cfs/cdirs/m3363/lbann/spack.git; . $SPACK_ROOT/share/spack/setup-env.sh
spack env activate -p lbann-dev-skylake_avx512

module use /global/cfs/cdirs/m3363/lb

In [25]:
### Move to staging locations in project space:
os.chdir(staging_loc)

### Submit jobs to cori queue

In [26]:
%%bash -s "$filename" ## Use python variable in bash
module load esslurm
sbatch $1

Submitted batch job 1251983
